In [313]:
import cv2
import os
import numpy as np
from collections import Counter
from skimage.color import rgb2lab

In [314]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [315]:
# def find_highest_rgb_value(image_path):
#     # Read the image
#     img = cv2.imread(image_path)

#     # Convert the image to RGB (OpenCV uses BGR by default)
#     img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#     # Find the pixel with the maximum RGB values
#     max_rgb_pixel = np.amax(img_rgb, axis=(0, 1))

#     return max_rgb_pixel

In [316]:
def calculate_average_cielab_exclude_zeros(image_path):
    # Read the image
    img = cv2.imread(image_path)

    # Convert the image to LAB color space
    img_lab = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)

    # Exclude pixels with minimum L channel values (e.g., zeros)
    non_zero_pixels = img_lab[img_lab[..., 0] != 0]

    # Check if there are non-zero pixels
    if len(non_zero_pixels) > 0:
        # Calculate the average LAB values for non-zero pixels
        average_lab = np.mean(non_zero_pixels, axis=0)
    else:
        # If all pixels are zeros, set the average to zeros
        average_lab = np.zeros(3)

    return average_lab

In [317]:
# def find_most_occurring_rgb_values(image_path, limit=10):
#     # Read the image
#     img = cv2.imread(image_path)

#     # Convert the image to RGB (OpenCV uses BGR by default)
#     img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#     # Flatten the 2D array to a list of RGB values
#     rgb_values = img_rgb.reshape((-1, 3))

#     # Use Counter to count occurrences of RGB values
#     rgb_counter = Counter(map(tuple, rgb_values))

#     # Get the most occurring RGB values within the specified limit
#     most_occurring_rgb_values = [list(rgb) for rgb, _ in rgb_counter.most_common(limit)]

#     average_rgb = np.mean(most_occurring_rgb_values, axis=0)

#     return average_rgb

In [318]:
def calculate_average_rgb_for_folder(folder_path):
    # Get a list of all files in the folder
    image_files = [f for f in os.listdir(folder_path) if f.endswith('.jpg') or f.endswith('.png')]

    # Create a dictionary to store average RGB values along with image names
    average_rgb_dict = {}

    # Calculate average RGB for each image
    for image_file in image_files:
        image_path = os.path.join(folder_path, image_file)
        average_rgb = calculate_average_rgb_exclude_zeros(image_path)

        # Store average RGB values along with image name in the dictionary
        average_rgb_dict[image_file] = average_rgb

    return average_rgb_dict

In [319]:
# Replace 'your_folder_path' with the path to your folder containing cropped images
folder_path = '/content/drive/My Drive/CustomObjDet/data/runs/detect/out_image_smoothed/'
result_dict = calculate_average_rgb_for_folder(folder_path)

# Print or use the result_dict as needed
for image_name, average_rgb in result_dict.items():
    print(f"Image: {image_name}, Average RGB values: {average_rgb}")

Image: 11.jpg, Average RGB values: [215.21316614 202.55799373 158.33855799]
Image: 12.jpg, Average RGB values: [141.19122257 194.24137931 169.3354232 ]
Image: 13.jpg, Average RGB values: [102.87175325 111.67045455  91.23538961]
Image: 10.jpg, Average RGB values: [216.34006211 208.32453416 156.40217391]
Image: 9.jpg, Average RGB values: [217.02998501 193.49625187 119.77811094]
Image: 5.jpg, Average RGB values: [215.57954545 193.33279221 149.51461039]
Image: 1.jpg, Average RGB values: [160.77714286 158.96571429 115.93142857]
Image: 3.jpg, Average RGB values: [200.25       164.91666667  54.73809524]
Image: 7.jpg, Average RGB values: [142.18013468 153.496633    56.86363636]
Image: 6.jpg, Average RGB values: [166.36490683 165.28726708  75.40838509]
Image: 2.jpg, Average RGB values: [223.66304348 170.14130435  66.67546584]
Image: 4.jpg, Average RGB values: [218.96296296 211.56902357 162.42760943]
Image: 8.jpg, Average RGB values: [225.80279503 219.27484472 178.98913043]
Image: 0.jpg, Average

In [320]:
# Replace 'your_folder_path' with the path to your folder containing cropped images
folder_path2 = '/content/drive/My Drive/CustomObjDet/data/runs/detect/ref_img_smoothed/'
result_dict_ref = calculate_average_rgb_for_folder(folder_path2)

# Print or use the result_dict as needed
for image_name, average_rgb in result_dict_ref.items():
    print(f"Image: {image_name}, Average RGB values: {average_rgb}")

Image: 24.jpg, Average RGB values: [163.90033223 134.03026947  97.60428202]
Image: 23.jpg, Average RGB values: [114.59928315  91.59928315  83.59928315]
Image: 21.jpg, Average RGB values: [162.44513458 109.44513458  65.3347136 ]
Image: 22.jpg, Average RGB values: [153.50508475 171.50508475 147.50508475]
Image: 20.jpg, Average RGB values: [ 74.84927536  90.27681159 107.9942029 ]
Image: 19.jpg, Average RGB values: [132.52230611 161.99794097 149.99794097]
Image: 6.jpg, Average RGB values: [148.15231458 127.16426083  97.84967646]
Image: 10.jpg, Average RGB values: [194.25555894 149.3347548  143.31647883]
Image: 8.jpg, Average RGB values: [111.90251359 108.03023098  96.41745924]
Image: 9.jpg, Average RGB values: [102.76961436 100.83942819 103.84973404]
Image: 7.jpg, Average RGB values: [179. 129. 140.]
Image: 12.jpg, Average RGB values: [89.767991 92.767991 75.767991]
Image: 18.jpg, Average RGB values: [104.21466974 124.74423963 121.5437788 ]
Image: 13.jpg, Average RGB values: [202.94037037 

In [321]:
def calculate_distance_lab(rgb1, rgb2):
    # Convert RGB to LAB color space
    lab1 = rgb2lab(rgb1[np.newaxis, np.newaxis, :] / 255.0).reshape(-1, 3)
    lab2 = rgb2lab(rgb2[np.newaxis, np.newaxis, :] / 255.0).reshape(-1, 3)

    # Calculate Euclidean distance in LAB color space
    distance = np.linalg.norm(lab1 - lab2, axis=-1)

    return np.mean(distance)

def find_best_matching_color(dict1, dict2, source_folder, output_folder):
    for image_name1, rgb_dict1 in dict1.items():
        best_match_image_name2 = None
        best_distance = float('inf')

        for image_name2, rgb_dict2 in dict2.items():
            # Calculate distance in LAB color space
            distance = calculate_distance_lab(rgb_dict1, rgb_dict2)

            # Update best match if the current distance is smaller
            if distance < best_distance:
                best_distance = distance
                best_match_image_name2 = image_name2


        print(f"Best match for {image_name1}: {best_match_image_name2} (Distance: {best_distance})")

        base_name_image_name1, _ = os.path.splitext(image_name1)

        # You can save or perform other actions here
        best_matched_image_path = os.path.join(source_folder, best_match_image_name2)
        best_matched_image = cv2.imread(best_matched_image_path)
        cv2.imwrite(os.path.join(output_folder, f"{base_name_image_name1}_{best_match_image_name2}"), best_matched_image)


In [322]:
source_folder = '/content/drive/My Drive/CustomObjDet/data/runs/detect/ref_image_smoothed/'
output_folder = '/content/drive/My Drive/CustomObjDet/data/runs/detect/out_image_smoothed/'
find_best_matching_color(result_dict, result_dict_ref, source_folder, output_folder)

Best match for 11.jpg: 13.jpg (Distance: 14.314867950053667)
Best match for 12.jpg: 22.jpg (Distance: 13.580985491550011)
Best match for 13.jpg: 8.jpg (Distance: 7.435792660307588)
Best match for 10.jpg: 13.jpg (Distance: 18.711208534726566)
Best match for 9.jpg: 26.jpg (Distance: 16.257041334836497)
Best match for 5.jpg: 13.jpg (Distance: 12.105333953030923)
Best match for 1.jpg: 33.jpg (Distance: 12.077850505495983)
Best match for 3.jpg: 26.jpg (Distance: 11.964454102633125)
Best match for 7.jpg: 37.jpg (Distance: 11.603200460623674)
Best match for 6.jpg: 37.jpg (Distance: 9.539877107096187)
Best match for 2.jpg: 26.jpg (Distance: 15.410687711601357)
Best match for 4.jpg: 13.jpg (Distance: 18.34243537480575)
Best match for 8.jpg: 13.jpg (Distance: 17.645299423339626)
Best match for 0.jpg: 10.jpg (Distance: 4.352382305549247)
